In [91]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [92]:
orig_data = pd.DataFrame(json_data)
orig_data = orig_data.drop(['tags', 'plylst_title', 'updt_date'], axis=1)
orig_data.head()

,id,songs,like_cnt
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...",1
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...",17
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...",33
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [96]:
orig_data.shape

(115071, 4)

In [37]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [38]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['issue_date', 'song_name', 'artist_id_basket', 'album_name', 'artist_name_basket', 'song_gn_gnr_basket'], axis=1)
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket':'genre'}, inplace=True)
song_data.head()

,genre,album_id,song_id
0,[GN0901],2255639,0
1,"[GN1601, GN1606]",376431,1
2,[GN0901],4698747,2
3,"[GN1102, GN1101]",2644882,3
4,"[GN1802, GN1801]",2008470,4


In [39]:
genre_list = []

for l in song_data['genre']:
    temp = dict()
    g = 0
    mx = 0
    
    for i in l:
        i = i[2:]
        n = int(i) // 100
        if n in temp:
            temp[n] += 1
        else:
            temp[n] = 1
    
    for i in temp:
        if mx < temp[i]:
            g = i
            mx = temp[i]
        elif mx == temp[i] and g > i:
            g = i
        
    genre_list.append(g)

song_data['genre'] = genre_list
song_data.head()

,genre,album_id,song_id
0,9,2255639,0
1,16,376431,1
2,9,4698747,2
3,11,2644882,3
4,18,2008470,4


In [56]:
with open('../Datasets/test.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [57]:
test_data = pd.DataFrame(json_data)
test_data = test_data.drop(['tags', 'plylst_title', 'updt_date'], axis=1)
test_data.head()

,id,songs,like_cnt
0,70107,"[398985, 449403, 411543, 528044, 143048, 98020...",6
1,7461,"[196298, 269984, 267805, 175867, 529244, 63825...",0
2,90348,"[273433, 331003, 68432, 411659, 117793, 616860...",21
3,58617,"[702227, 48152, 440008, 358488, 701041, 540721...",0
4,102395,"[630683, 481582, 528550, 285114, 506667, 17922...",38


In [58]:
test_data.rename(columns={'songs':'song_id'}, inplace=True)
test_data.head()

,id,song_id,like_cnt
0,70107,"[398985, 449403, 411543, 528044, 143048, 98020...",6
1,7461,"[196298, 269984, 267805, 175867, 529244, 63825...",0
2,90348,"[273433, 331003, 68432, 411659, 117793, 616860...",21
3,58617,"[702227, 48152, 440008, 358488, 701041, 540721...",0
4,102395,"[630683, 481582, 528550, 285114, 506667, 17922...",38


# BorderlineSMOTE

- 너무 많은 데이터로 인해 1000개의 플레이리스트를 뽑아와 oversampling
- 1000개의 플레이리스트를 oversampling한 결과(45196 -> 113467)
- 상당히 과적합으로 보임

In [71]:
orig_data_sample = orig_data.loc[:1000]
orig_data_sample = orig_data_sample.explode('songs')
orig_data_sample.rename(columns={'songs':'song_id'}, inplace=True)
orig_data_sample.head()

,id,song_id,like_cnt
0,61281,525514,71
0,61281,129701,71
0,61281,383374,71
0,61281,562083,71
0,61281,297861,71


In [72]:
orig_data_sample.shape

(45196, 3)

In [45]:
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE

sm_x = orig_data_sample.drop(['id'], axis=1)
sm_y = orig_data_sample['id']

print(Counter(sm_y))

sm = BorderlineSMOTE(random_state=0, k_neighbors=4)
sm_x_res, sm_y_res = sm.fit_resample(sm_x, sm_y)

print(Counter(sm_y_res))
print(sm_x_res.shape)

Counter({131098: 200, 8455: 200, 80395: 200, 31654: 200, 114074: 200, 80036: 200, 27272: 200, 27187: 200, 9180: 200, 141805: 200, 131918: 200, 112794: 200, 42641: 200, 106802: 200, 37260: 200, 110348: 200, 141844: 200, 109650: 200, 127865: 200, 100254: 200, 78149: 200, 67939: 200, 98953: 200, 67951: 200, 50553: 200, 49329: 200, 64447: 200, 70033: 200, 10987: 200, 12221: 200, 68338: 200, 121021: 200, 27449: 200, 2406: 200, 39348: 200, 147065: 197, 145880: 190, 89940: 189, 55459: 187, 95004: 182, 9323: 182, 21520: 179, 3380: 178, 100434: 172, 6762: 171, 32224: 168, 20234: 164, 18218: 163, 6393: 161, 109474: 160, 149053: 159, 31804: 157, 134754: 154, 37064: 150, 5069: 147, 8965: 145, 12066: 144, 50790: 142, 145624: 140, 16182: 135, 131403: 134, 92125: 130, 66532: 129, 45306: 129, 112304: 127, 13041: 126, 50705: 125, 60068: 125, 123055: 123, 120983: 120, 152705: 120, 72590: 118, 19592: 118, 116581: 116, 52701: 116, 112218: 115, 146980: 114, 115660: 114, 15011: 113, 18865: 109, 93888: 107, 

Counter({10532: 200, 76951: 200, 36557: 200, 70741: 200, 10288: 200, 31804: 200, 112060: 200, 89809: 200, 8316: 200, 1516: 200, 5348: 200, 88143: 200, 62696: 200, 136677: 200, 144318: 200, 93836: 200, 43665: 200, 35178: 200, 131098: 200, 61024: 200, 82383: 200, 63703: 200, 137737: 200, 67565: 200, 16037: 200, 137153: 200, 33019: 200, 124665: 200, 118405: 200, 26774: 200, 12956: 200, 141483: 200, 97458: 200, 60192: 200, 95004: 200, 19726: 200, 60834: 200, 98228: 200, 8455: 200, 110924: 200, 93434: 200, 34226: 200, 34198: 200, 140334: 200, 61655: 200, 33025: 200, 28452: 200, 116581: 200, 12743: 200, 132983: 200, 143459: 200, 150721: 200, 18696: 200, 60811: 200, 119639: 200, 134754: 200, 37306: 200, 13366: 200, 15630: 200, 96846: 200, 16542: 200, 1514: 200, 115374: 200, 43222: 200, 50570: 200, 52767: 200, 32533: 200, 140912: 200, 73145: 200, 112195: 200, 10705: 200, 80052: 200, 92503: 200, 130786: 200, 41361: 200, 10901: 200, 133097: 200, 60844: 200, 125058: 200, 51730: 200, 72590: 200, 2

In [46]:
sm_x_res.head(100)

,song_id,like_cnt
0,525514.0,71
1,129701.0,71
2,383374.0,71
3,562083.0,71
4,297861.0,71
...,...,...
95,556301.0,33
96,655561.0,33
97,534818.0,33
98,695032.0,33


In [51]:
sm_x_res['song_id'] = sm_x_res['song_id'] // 1
sm_x_res = sm_x_res.astype('int')
sm_x_res.head(100)

,song_id,like_cnt
0,525514,71
1,129701,71
2,383374,71
3,562083,71
4,297861,71
...,...,...
95,556301,33
96,655561,33
97,534818,33
98,695032,33


In [53]:
merge_sm = pd.merge(sm_x_res, song_data)
merge_sm.head()

,song_id,like_cnt,genre,album_id
0,525514,71,14,2200223
1,129701,71,9,2201802
2,383374,71,10,2216938
3,562083,71,9,43227
4,297861,71,9,303657


In [55]:
from sklearn.neighbors import KNeighborsClassifier 
kn_sm = KNeighborsClassifier(n_neighbors=1)

kn_sm.fit(merge_sm, sm_y_res)
print(kn_sm.score(merge_sm, sm_y_res))

0.9985987115196489


In [90]:
sm_data = pd.concat([merge_sm, sm_y_res], axis=1)
sm_data.head(100)

,song_id,like_cnt,genre,album_id,id
0,525514,71,14,2200223,61281
1,129701,71,9,2201802,61281
2,383374,71,10,2216938,61281
3,562083,71,9,43227,61281
4,297861,71,9,303657,61281
...,...,...,...,...,...
95,152422,0,1,2282532,147456
96,152422,0,1,2282532,147456
97,152422,7,1,2282532,147456
98,152422,7,1,2282532,147456


In [65]:
import numpy as np

for ti in test_data['id']:
    test_temp = test_data[test_data['id'] == ti]
    
    test_temp = test_temp.explode('song_id')
    
    test_temp_merge = pd.merge(test_temp, song_data)
    test_src = test_temp_merge.drop(['id'], axis=1)
    
    if test_src.empty:
        continue
    
    id_list = kn_sm.predict(test_src)
    
    id_nums = dict()
    
    for i in id_list:
        if i in id_nums:
            id_nums[i] += 1
        else:
            id_nums[i] = 1
    
    ids = sorted(id_nums.items(), key=lambda x: x[1], reverse=True)
    ids_index = 0
    cnt = 0
    
    recommend = []
    
    while cnt < 30 and len(ids) > ids_index:
        for song in sm_data[sm_data['id']==ids[ids_index][0]]['song_id']:
            if not song in test_temp['song_id'] and not song in recommend:
                recommend.append(song)
                cnt += 1
            
            if cnt == 30:
                break
        
        ids_index += 1
    
    print(recommend)

[194542, 148992, 76676, 282168, 131310, 205255, 168353, 466272, 554751, 344769, 678423, 542776, 669027, 671873, 183173, 180280, 537730, 666361, 615236, 581556, 168984, 669453, 544537, 542512, 542667, 25329, 205559, 15327, 46627, 653646]
[681062, 540822, 565196, 565698, 575795, 440779, 405204, 542770, 52626, 65980, 71313, 29624, 439630, 631242, 44668, 269483, 269233, 604560, 661966, 98586, 500751, 177909, 286441, 158289, 637866, 496560, 361240, 553018, 361425, 14316]
[86533, 122836, 124278, 597627, 273433, 686295, 289687, 645990, 136458, 583448, 378627, 324948, 6394, 18821, 41543, 47106, 255469, 86640, 319861, 330740, 188271, 233708, 283895, 527255, 659122, 526963, 368055, 217739, 585053, 598097]
[214891, 376360, 196614, 231791, 106876, 422812, 334672, 14212, 584537, 656832, 56325, 93461, 452637, 499969, 163310, 198982, 545231, 76424, 599676, 274608, 624719, 244008, 188030, 302868, 646751, 142059, 92876, 489138, 368170, 692638]
[122182, 355145, 475313, 471988, 470535, 419853, 596794, 31

[206369, 232046, 421613, 507650, 429671, 399591, 693354, 471515, 537999, 41065, 554292, 120364, 663831, 201587, 319535, 430201, 637799, 257372, 641954, 177126, 402899, 661148, 643821, 347191, 648132, 339988, 342255, 222892, 356379, 354676]
[102206, 361178, 175265, 21690, 476707, 113847, 31258, 547185, 27784, 140749, 421743, 691570, 264929, 483384, 364040, 223411, 338856, 223821, 198358, 282417, 53708, 369900, 83955, 181812, 11908, 205969, 108512, 667098, 579675, 394061]
[251375, 7766, 704752, 580326, 646941, 699557, 157435, 544308, 603044, 347585, 468830, 473385, 476502, 473454, 473275, 473356, 476797, 473432, 473495, 473413, 484233, 467914, 476303, 473490, 473329, 473478, 488894, 473198, 462623, 467848]
[468206, 673851, 433118, 10969, 209932, 524210, 699497, 193269, 471037, 451233, 224130, 436458, 303171, 500484, 223327, 343917, 621707, 25799, 505652, 8659, 582747, 444095, 77783, 657184, 80317, 703820, 645601, 637048, 443907, 443904]
[365133, 543675, 183468, 626851, 294481, 294240, 68

KeyboardInterrupt: 

# ADASYN

- 플레이리스트 내의 노래가 너무 적어 oversampling을 할 수 없음
- 너무 적은 플레이리스트를 제거한 후 수행(중요한 데이터가 손실될 수도 있으나 시도)
- 기준 : 5개 이하의 노래를 가진 플레이리스트 제거
- 제거한 후 1000개의 플레이리스트 추출(oversampling 시에 상당한 시간이 소요)
- 옵션을 sampling_strategy='minority' 로 하지 않는 경우 에러(ValueError: No samples will be generated with the provided ratio settings.) 발생
- sampling_strategy='minority' 옵션 선택 시 만족스러울 정도의 oversampling이 되지 않음(45175 -> 45367)

In [93]:
ad_pre_data = orig_data

lens = []
for l in ad_pre_data['songs']:
    lens.append(len(l))

ad_pre_data['len'] = lens
ad_pre_data.head()

,id,songs,like_cnt,len
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...",71,19
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...",1,42
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...",17,28
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...",33,38
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,53


In [99]:
ad_pre_data = ad_pre_data[ad_pre_data['len'] > 5]
ad_pre_data = ad_pre_data.drop(['len'], axis=1)
ad_pre_data.shape

(114375, 3)

In [103]:
ad_pre_data_sample = ad_pre_data.loc[:1000]
ad_pre_data_sample = ad_pre_data_sample.explode('songs')
ad_pre_data_sample.rename(columns={'songs':'song_id'}, inplace=True)
ad_pre_data_sample.head()

,id,song_id,like_cnt
0,61281,525514,71
0,61281,129701,71
0,61281,383374,71
0,61281,562083,71
0,61281,297861,71


In [104]:
ad_pre_data_sample.shape

(45175, 3)

In [110]:
from imblearn.over_sampling import ADASYN

ad_x = ad_pre_data_sample.drop(['id'], axis=1)
ad_y = ad_pre_data_sample['id']
print(Counter(ad_y))
print()

ad = ADASYN(random_state=0, sampling_strategy='minority')
ad_x_res, ad_y_res = ad.fit_resample(ad_x, ad_y)

print(Counter(ad_y_res))
print(ad_x_res.shape)

Counter({131098: 200, 8455: 200, 80395: 200, 31654: 200, 114074: 200, 80036: 200, 27272: 200, 27187: 200, 9180: 200, 141805: 200, 131918: 200, 112794: 200, 42641: 200, 106802: 200, 37260: 200, 110348: 200, 141844: 200, 109650: 200, 127865: 200, 100254: 200, 78149: 200, 67939: 200, 98953: 200, 67951: 200, 50553: 200, 49329: 200, 64447: 200, 70033: 200, 10987: 200, 12221: 200, 68338: 200, 121021: 200, 27449: 200, 2406: 200, 39348: 200, 147065: 197, 145880: 190, 89940: 189, 55459: 187, 95004: 182, 9323: 182, 21520: 179, 3380: 178, 100434: 172, 6762: 171, 32224: 168, 20234: 164, 18218: 163, 6393: 161, 109474: 160, 149053: 159, 31804: 157, 134754: 154, 37064: 150, 5069: 147, 8965: 145, 12066: 144, 50790: 142, 145624: 140, 16182: 135, 131403: 134, 92125: 130, 66532: 129, 45306: 129, 112304: 127, 13041: 126, 50705: 125, 60068: 125, 123055: 123, 120983: 120, 152705: 120, 72590: 118, 19592: 118, 116581: 116, 52701: 116, 112218: 115, 146980: 114, 115660: 114, 15011: 113, 18865: 109, 93888: 107, 

Counter({131098: 200, 8455: 200, 80395: 200, 31654: 200, 114074: 200, 80036: 200, 27272: 200, 27187: 200, 9180: 200, 141805: 200, 131918: 200, 112794: 200, 42641: 200, 106802: 200, 37260: 200, 110348: 200, 141844: 200, 109650: 200, 127865: 200, 100254: 200, 78149: 200, 67939: 200, 98953: 200, 67951: 200, 50553: 200, 49329: 200, 64447: 200, 70033: 200, 10987: 200, 12221: 200, 68338: 200, 121021: 200, 27449: 200, 2406: 200, 39348: 200, 41471: 198, 147065: 197, 145880: 190, 89940: 189, 55459: 187, 95004: 182, 9323: 182, 21520: 179, 3380: 178, 100434: 172, 6762: 171, 32224: 168, 20234: 164, 18218: 163, 6393: 161, 109474: 160, 149053: 159, 31804: 157, 134754: 154, 37064: 150, 5069: 147, 8965: 145, 12066: 144, 50790: 142, 145624: 140, 16182: 135, 131403: 134, 92125: 130, 66532: 129, 45306: 129, 112304: 127, 13041: 126, 50705: 125, 60068: 125, 123055: 123, 120983: 120, 152705: 120, 72590: 118, 19592: 118, 116581: 116, 52701: 116, 112218: 115, 146980: 114, 115660: 114, 15011: 113, 18865: 109, 

# OneSidedSelection

- undersampling을 하지만 소수 클래스에 해당하는 플레이리스트의 노래들이 너무 적어 데이터를 너무 삭제함
- 너무 적은 플레이리스트를 제거한 후 수행(중요한 데이터가 손실될 수도 있으나 시도)
- 기준 : 50개 이하의 노래를 가진 플레이리스트 제거

In [116]:
oss_pre_data = orig_data

lens = []
for l in oss_pre_data['songs']:
    lens.append(len(l))

oss_pre_data['len'] = lens
oss_pre_data.head()

,id,songs,like_cnt,len
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...",71,19
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...",1,42
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...",17,28
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...",33,38
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,53


In [117]:
oss_pre_data = oss_pre_data[oss_pre_data['len'] > 50]
oss_pre_data = oss_pre_data.drop(['len'], axis=1)
oss_pre_data.shape

(30952, 3)

In [118]:
oss_pre_data = oss_pre_data.explode('songs')
oss_pre_data.rename(columns={'songs':'song_id'}, inplace=True)
oss_pre_data.head()

,id,song_id,like_cnt
4,27616,159327,9
4,27616,553610,9
4,27616,5130,9
4,27616,645103,9
4,27616,294435,9


In [119]:
oss_pre_data.shape

(3190025, 3)

In [120]:
from imblearn.under_sampling import OneSidedSelection

oss_x = oss_pre_data.drop(['id'], axis=1)
oss_y = oss_pre_data['id']
print(Counter(oss_y))
print()

oss = OneSidedSelection(random_state=42)
oss_x_res, oss_y_res = oss.fit_resample(oss_x, oss_y)

print(Counter(oss_y_res))
print(oss_x_res.shape)

Counter({131098: 200, 8455: 200, 80395: 200, 31654: 200, 114074: 200, 80036: 200, 27272: 200, 27187: 200, 9180: 200, 141805: 200, 131918: 200, 112794: 200, 42641: 200, 106802: 200, 37260: 200, 110348: 200, 141844: 200, 109650: 200, 127865: 200, 100254: 200, 78149: 200, 67939: 200, 98953: 200, 67951: 200, 50553: 200, 49329: 200, 64447: 200, 70033: 200, 10987: 200, 12221: 200, 68338: 200, 121021: 200, 27449: 200, 2406: 200, 39348: 200, 56558: 200, 150339: 200, 2466: 200, 10377: 200, 117266: 200, 44154: 200, 140052: 200, 69831: 200, 15920: 200, 87937: 200, 141642: 200, 24450: 200, 92621: 200, 98070: 200, 35635: 200, 10711: 200, 31526: 200, 19975: 200, 147340: 200, 13317: 200, 43939: 200, 51223: 200, 4057: 200, 36535: 200, 10564: 200, 95459: 200, 104213: 200, 108200: 200, 130107: 200, 83633: 200, 148884: 200, 13123: 200, 88773: 200, 119212: 200, 133659: 200, 9407: 200, 39799: 200, 48312: 200, 16737: 200, 122000: 200, 67397: 200, 68822: 200, 38705: 200, 64404: 200, 146160: 200, 79484: 200, 

Counter({99040: 200, 139359: 200, 145594: 200, 72331: 199, 150417: 199, 151782: 199, 8172: 198, 20488: 198, 36879: 198, 92726: 198, 38199: 197, 57260: 197, 64743: 197, 121771: 197, 125441: 197, 134164: 197, 144854: 197, 150542: 197, 4656: 196, 20579: 196, 33949: 196, 48880: 196, 69614: 196, 87915: 196, 99071: 196, 101117: 196, 108491: 196, 128268: 196, 129172: 196, 144928: 196, 2062: 195, 17743: 195, 41422: 195, 44585: 195, 52946: 195, 56028: 195, 68085: 195, 77952: 195, 82702: 195, 85878: 195, 115243: 195, 152216: 195, 153170: 195, 20610: 194, 24022: 194, 36404: 194, 40537: 194, 47907: 194, 50591: 194, 54072: 194, 55384: 194, 67794: 194, 81796: 194, 88435: 194, 95713: 194, 101925: 194, 106802: 194, 111960: 194, 126037: 194, 128977: 194, 143430: 194, 144379: 194, 153422: 194, 3050: 193, 35780: 193, 59357: 193, 60743: 193, 67453: 193, 76663: 193, 79556: 193, 79865: 193, 86438: 193, 95948: 193, 112601: 193, 119460: 193, 122103: 193, 126759: 193, 138784: 193, 140279: 193, 145086: 193, 216

In [129]:
oss_x_res = oss_x_res.astype('int')
oss_x_res.head(100)

,song_id,like_cnt
0,148188,180
1,379113,180
2,298644,180
3,14996,180
4,386572,180
...,...,...
95,629531,6
96,105407,6
97,367360,6
98,105451,6


In [130]:
merge_oss = pd.merge(oss_x_res, song_data)
merge_oss.head()

,song_id,like_cnt,genre,album_id
0,148188,180,11,2169521
1,148188,40,11,2169521
2,148188,7,11,2169521
3,148188,7,11,2169521
4,148188,20,11,2169521


In [131]:
kn_oss = KNeighborsClassifier(n_neighbors=1)

kn_oss.fit(merge_oss, oss_y_res)
print(kn_sm.score(merge_oss, oss_y_res))

3.6749455533847856e-06


In [132]:
oss_data = pd.concat([merge_oss, oss_y_res], axis=1)
oss_data.head(100)

,song_id,like_cnt,genre,album_id,id
0,148188,180,11,2169521,6
1,148188,40,11,2169521,6
2,148188,7,11,2169521,6
3,148188,7,11,2169521,6
4,148188,20,11,2169521,6
...,...,...,...,...,...
95,14996,35,11,2320200,12
96,14996,696,11,2320200,12
97,14996,527,11,2320200,12
98,386572,180,11,2704351,12


In [133]:
import numpy as np

for ti in test_data['id']:
    test_temp = test_data[test_data['id'] == ti]
    
    test_temp = test_temp.explode('song_id')
    
    test_temp_merge = pd.merge(test_temp, song_data)
    test_src = test_temp_merge.drop(['id'], axis=1)
    
    if test_src.empty:
        continue
    
    id_list = kn_oss.predict(test_src)
    
    id_nums = dict()
    
    for i in id_list:
        if i in id_nums:
            id_nums[i] += 1
        else:
            id_nums[i] = 1
    
    ids = sorted(id_nums.items(), key=lambda x: x[1], reverse=True)
    ids_index = 0
    cnt = 0
    
    recommend = []
    
    while cnt < 30 and len(ids) > ids_index:
        for song in oss_data[oss_data['id']==ids[ids_index][0]]['song_id']:
            if not song in test_temp['song_id'] and not song in recommend:
                recommend.append(song)
                cnt += 1
            
            if cnt == 30:
                break
        
        ids_index += 1
    
    print(recommend)

[398985, 89992, 448369, 449403, 617428, 679614, 411543, 553522, 265351, 528044, 283485, 265538, 143048, 320204, 329553, 642252, 195835, 585621, 613026, 514150, 395546, 98020, 686605, 17332, 649349, 231304, 393892, 251940, 1174, 697665]
[632613, 486127, 195824, 625982, 344420, 661734, 591748, 82466, 269984, 18057, 267805, 116711, 539630, 98486, 332364, 385173, 539747, 451588, 175772, 140722, 290552, 566673, 574195, 31244, 451503, 108301, 650237, 181106, 485411, 187222]
[356220, 273433, 665324, 331003, 462740, 68432, 289613, 685412, 61985, 211593, 411659, 258115, 448621, 320374, 563429, 117793, 273114, 616860, 551588, 482137, 97062, 701311, 246496, 72175, 119263, 515607, 290858, 112580, 300523, 81870]
[342195, 691005, 580868, 545963, 592500, 610909, 399967, 565769, 155387, 441805, 526461, 393417, 343049, 565683, 454355, 631642, 454158, 505472, 565247, 358297, 660726, 593702, 165654, 361448, 502427, 116683, 3926, 567970, 500038, 702227]
[221101, 639275, 323372, 45197, 514540, 533391, 5318

[275647, 300918, 373454, 515802, 146879, 119715, 63246, 433541, 28122, 152846, 176810, 148090, 496705, 508214, 629566, 95128, 171562, 339596, 646293, 253938, 94333, 565100, 439612, 214529, 138524, 422125, 690541, 42454, 670581, 640234]
[56352, 157435, 681234, 580677, 408758, 417368, 259795, 545585, 97942, 11657, 32330, 114259, 98480, 114387, 331794, 507394, 151218, 110510, 183690, 628491, 188196, 34363, 420396, 378627, 460784, 391271, 590029, 116645, 616798, 115351]
[92166, 469045, 699432, 681315, 458383, 297319, 125634, 158299, 181297, 430129, 94468, 283495, 680579, 656601, 86941, 501106, 499578, 528908, 114177, 382247, 659808, 494770, 254855, 440883, 600315, 369731, 120040, 136083, 704336, 568281]
[594590, 539509, 381842, 353139, 346340, 450005, 391069, 276616, 690554, 688517, 483907, 642845, 455583, 81264, 492258, 251136, 236402, 667161, 55838, 399211, 396033, 185692, 688620, 97227, 536619, 264837, 319780, 60864, 70926, 658071]
[129545, 122309, 112857, 675115, 657957, 333146, 184543

KeyboardInterrupt: 